# Цель:
Исследовать поведение пользователей в обновлённом приложении Quiz Freeze.

# Задачи:

**1.** Определите, насколько обучение сокращает время прохождения этапов игры.

**2.** Докажите, что успешное обучение само по себе влияет на оплату и не имеет значения то, каким этапом оно шло.

**3.** Проверьте, насколько прозрачен процесс взаимодействия с игрой.

# Проверить следующие гипотезы:

**Гипотеза 1:** Влияет ли обучение на скорость прохождения других этапов игры?

**Гипотеза 2:** Влияет ли на оплату повторное обучение?

**Гипотеза 3:** Если пользователь сначала выбирает сложность обучения, будет ли он потом проходить обучение?

# Предварительные шаги

Импортируем библиотеку и исходные данные.

In [ ]:
import pandas as pd
events_df = pd.read_csv('data/7_4_Events.csv', sep=',')
purchase_df = pd.read_csv('data/purchase.csv', sep=',')

Отфильтруем базы данных и оставим только пользователей зарегистрированных в 2018 году. Скорректируем форматы дат:

In [ ]:
# Преобразуем даты в формат datetime.
events_df['start_time'] = pd.to_datetime(events_df['start_time'], errors='coerce')
purchase_df['event_datetime'] = pd.to_datetime(purchase_df['event_datetime'])


# Cоздаем список пользователей, зарегистрировавшихся в 2018 году.
cond_1 = events_df['event_type'] == 'registration'
cond_2 = events_df['start_time'] >= '2018-01-01'
cond_3 = events_df['start_time'] < '2019-01-01'
registered_in_2018 = events_df[cond_1 & cond_2 & cond_3]['user_id'].to_list()


# Сставляем в таблицах только нужные записи.
events_df = events_df[events_df['user_id'].isin(registered_in_2018)]
purchase_df = purchase_df[purchase_df['user_id'].isin(registered_in_2018)]

Объединим датафреймы событий и оплат:

In [ ]:
# Добавим в датафрейм purchase_df столбец event_type, чтобы
# однозначно выделить события оплаты в объединенном датафрейме.
purchase_df['event_type'] = 'purchase'

# Переименуем столбцы с одинакомыми названиями.
events_df = events_df.rename(columns={'id': 'event_id'})
purchase_df = purchase_df.rename(columns={'id': 'purchase_id'})

# Также переименуем столбец start_time, чтобы время покупок и остальных событий были в одном столбце.
events_df = events_df.rename(columns={'start_time': 'event_datetime'})

#Объединими датафреймы.
total_events_df = pd.concat([events_df, purchase_df], sort=False)

#Сбросим индексы нового датафрейма и отсортируем события по времени.
total_events_df = total_events_df.reset_index(drop=True).sort_values('event_datetime')

total_events_df.head()

,event_id,event_type,selected_level,event_datetime,tutorial_id,user_id,purchase_id,amount
0,80308.0,registration,NaN,2018-01-01 03:48:40,NaN,27832,NaN,NaN
1,80309.0,registration,NaN,2018-01-01 04:07:25,NaN,27833,NaN,NaN
2,80310.0,registration,NaN,2018-01-01 08:35:10,NaN,27834,NaN,NaN
3,80311.0,registration,NaN,2018-01-01 11:54:47,NaN,27835,NaN,NaN
4,80312.0,registration,NaN,2018-01-01 13:28:07,NaN,27836,NaN,NaN


# Проверка гипотезы 1.

***Влияет ли обучение на скорость прохождения других этапов игры?***

Чтобы узнать это, необходимо сравнить время прохождения различных этапов для пользователей, которые завершили обучение, и пользователей, не начинавших обучение. Если показатель отличается, выяснить насколько.


In [ ]:
# Найдем группу пользователей, завершивших обучение.
users_finished_tutorial = total_events_df[
    total_events_df['event_type'] == 'tutorial_finish']['user_id'].unique()
set_users_finished_tutorial = set(users_finished_tutorial)

# Найдем группу пользователей, не начинавших обучение.
# Для этого вычтем из всех пользователей тех, кто начинал обучение.
all_users = total_events_df['user_id'].unique()
set_all_users = set(all_users)

users_started_tutorial = total_events_df[
    total_events_df['event_type'] == 'tutorial_start']['user_id'].unique()
set_users_started_tutorial = set(users_started_tutorial)

set_users_without_tutorial = set_all_users.difference(set_users_started_tutorial)

In [ ]:

# Создадим функцию, определяющую к какой группе относится пользователь.
def identificate_group(user):
    if user in set_users_finished_tutorial:
        return 'finished_tutorial'
    elif user in set_users_without_tutorial:
        return 'without_tutorial'
    else:
        return 'other'

# Применим эту функцию, чтобы создать в датафрейме новый признак.
total_events_df['user_group'] = total_events_df['user_id'].apply(identificate_group)

total_events_df.head()

,event_id,event_type,selected_level,event_datetime,tutorial_id,user_id,purchase_id,amount,user_group
0,80308.0,registration,NaN,2018-01-01 03:48:40,NaN,27832,NaN,NaN,without_tutorial
1,80309.0,registration,NaN,2018-01-01 04:07:25,NaN,27833,NaN,NaN,finished_tutorial
2,80310.0,registration,NaN,2018-01-01 08:35:10,NaN,27834,NaN,NaN,finished_tutorial
3,80311.0,registration,NaN,2018-01-01 11:54:47,NaN,27835,NaN,NaN,finished_tutorial
4,80312.0,registration,NaN,2018-01-01 13:28:07,NaN,27836,NaN,NaN,finished_tutorial


In [ ]:
# С помощью сводной таблицы найдем, когда пользователи впервые переходили на каждый этап.
users_events_datetime = total_events_df.pivot_table(
    values='event_datetime',
    index='user_id',
    columns='event_type',
    aggfunc='min'
)

In [ ]:
# Найдем длительность перехода на каждый этап.
events = ['tutorial_start', 'tutorial_finish', 'level_choice', 'pack_choice', 'purchase']
for event in events:
    users_events_datetime['td_{}'.format(event)] = users_events_datetime[event] - users_events_datetime['registration']

# Уберем лишние столбцы.
users_events_td = users_events_datetime.drop(
    ['registration', 'tutorial_start', 'tutorial_finish', 'level_choice', 'pack_choice', 'purchase'], axis=1)

In [ ]:
# Теперь построим окончательную сводную таблицу, чтобы оценить интервалы для групп пользователей.
# Сначала добавим столбцы user_id и user_group.
users_events_td['user_id'] = users_events_datetime.index
users_events_td['user_group'] = users_events_td['user_id'].apply(identificate_group)

# Создадим саму сводную таблицу.
groups_events_td = users_events_td.groupby(by='user_group')[
    ['td_tutorial_start', 'td_tutorial_finish', 'td_level_choice', 'td_pack_choice', 'td_purchase']
].mean()

groups_events_td = groups_events_td.drop('other')

display(groups_events_td)

event_type,td_tutorial_start,td_tutorial_finish,td_level_choice,td_pack_choice,td_purchase
user_group,,,,,
finished_tutorial,0 days 04:31:04.938146341,0 days 04:43:18.761268292,0 days 07:05:36.854819357,0 days 07:10:35.660162287,4 days 00:30:03.247408431
without_tutorial,NaT,NaT,0 days 05:15:33.122448979,0 days 05:07:16.175675675,4 days 09:06:01.909090909


In [ ]:
# Сначала необходимо преобразовать тип данных в сводной таблице из временного в числовой.
# За единицу измерения интервалов выберем час.
groups_events_hours_td = round(groups_events_td/pd.Timedelta('1 hour'), 2)
groups_events_hours_td['user_group'] = groups_events_hours_td.index
groups_events_hours_td = groups_events_hours_td.fillna(0)

display(groups_events_hours_td)

event_type,td_tutorial_start,td_tutorial_finish,td_level_choice,td_pack_choice,td_purchase,user_group
user_group,,,,,,
finished_tutorial,4.52,4.72,7.09,7.18,96.5,finished_tutorial
without_tutorial,0.00,0.00,5.26,5.12,105.1,without_tutorial


In [ ]:
# Импортируем библиотеку.
import plotly
import plotly.express as px

# Строим график.
fig1 = px.histogram(
    data_frame=groups_events_hours_td,
    x='user_group',
    y=['td_tutorial_start', 'td_tutorial_finish', 'td_level_choice', 'td_pack_choice', 'td_purchase'],
    title='Later in hour',
    # color='user_group',
    width=700,
    height=700,
    #notched=True,
    #points="all"
)
fig1.show()

In [ ]:
# Посчитаем разницу во времени в процентом соотношении для пути от регистрации до покупка платных пакетов вопросов
round(((groups_events_hours_td.iloc[1][4] - groups_events_hours_td.iloc[0][4]) / groups_events_hours_td.iloc[1][4]) * 100, 1)

8.2

Вывод: у группы без обучения этапы короче чем у группы с обучением за счет времени самого обучения, однако, если посмотреть весь путь от регистрации до покупки платных пакетов вопросов, то он для группы не прошедших обучение занимает времени на 8,2% больше.

# Проверка гипотезы 2.

***Влияет ли на оплату повторное обучение?***

Нужно проверить, существует ли зависимость между вероятностью оплаты вопросов и количеством обучений, которые начинал или завершал пользователь.

In [ ]:
#Создадим список пользователей совершивших покупку
set_users_with_purchase = set(total_events_df[total_events_df.event_type == 'purchase']['user_id'].unique())

In [ ]:
#Создадим датафрейм с количеством попыток прохождения обучения пользователями
tutorial_attempts_df = total_events_df.groupby('user_id')['tutorial_id'].nunique().reset_index()
tutorial_attempts_df = tutorial_attempts_df.rename(columns={'tutorial_id':'tutorial_attempts'})

In [ ]:
#Добавим столбец purchase, польователи без покупки будут со значением 0
tutorial_attempts_df['purchase'] = tutorial_attempts_df.apply(
    lambda x : 1 if x ['user_id'] in set_users_with_purchase else 0, axis=1
)

In [ ]:
# Найдем длительность перехода на каждый этап.
events = ['tutorial_start', 'tutorial_finish', 'level_choice', 'pack_choice', 'purchase']
for event in events:
    users_events_datetime['td_{}'.format(event)] = users_events_datetime[event] - users_events_datetime['registration']

In [ ]:
#Сгруппируем по количеству прохождения обучения пользователями и посчитаем колличество покупок
tutorial_attempts_df.loc[tutorial_attempts_df['tutorial_attempts'] > 3, 'tutorial_attempts'] = '4 and more attempts'
count_tutorial_attempts_df = tutorial_attempts_df.groupby('tutorial_attempts')['purchase'].sum().reset_index()
count_tutorial_attempts_df.loc[count_tutorial_attempts_df['tutorial_attempts'] == 3, 'tutorial_attempts'] = '3 attempts'
count_tutorial_attempts_df.loc[count_tutorial_attempts_df['tutorial_attempts'] == 2, 'tutorial_attempts'] = '2 attempts'
count_tutorial_attempts_df.loc[count_tutorial_attempts_df['tutorial_attempts'] == 1, 'tutorial_attempts'] = '1 attempt'
count_tutorial_attempts_df.loc[count_tutorial_attempts_df['tutorial_attempts'] == 0, 'tutorial_attempts'] = '0 attempts'

In [ ]:
fig2 = px.pie(
    data_frame=count_tutorial_attempts_df,
    values='purchase',
    names='tutorial_attempts',
    height=500,
    width=700,
    title='Ratio of purchase'
)
fig2.show()

Вывод: попытки пройти обучение положительно вляиет на вероятность покупки платных вопросов, 98.6% пользователей совершие покупку платных вопросов начинали обучение хотя бы раз. Количество попыток обучения не влияет на вероятность покупки, 75.4% купили платный пакет полсе первого обучения, далее с увеличением колличества попыток наблюдается снижения доли из общего числа пользователей совершивших покупку.

# Проверка гипотезы 3.

***Если пользователь сначала выбирает сложность обучения, будет ли он потом проходить обучение?***

Выясним, как часто пользователи начинают обучение после выбора уровня сложности.

In [ ]:
# Создадим два датафрема с событием начала обучения(tutorial_start_df) и изменением уровня сложности(level_choice_df),
# переименуем столбцы с одинакомыми названиями
tutorial_start_df = events_df[events_df['event_type']=='tutorial_start'].rename(columns={'event_datetime':'tutorial_start_datetime'})
level_choice_df = events_df[events_df['event_type']=='level_choice'].rename(columns={'event_datetime':'level_choice_datetime'})

#Объединими датафреймы.
focus_group_df = pd.merge(level_choice_df, tutorial_start_df, on='user_id')

In [ ]:
#Добавим столбец timedelta с разницей во времен от начала обучения изменения уровня сложности
focus_group_df['timedelta'] = (focus_group_df['tutorial_start_datetime']-focus_group_df['level_choice_datetime'])
#Оставим тольке тех кто начал обучение после изменения уровеня, а так же сбросим индексы датафрейма
focus_group_df = focus_group_df[focus_group_df.timedelta>pd.Timedelta(0)].reset_index(drop=True)

In [ ]:
#Посчитаем долю пользователей, которые прошли обучение после выбора уровня сложности
round(((focus_group_df.user_id.nunique()/level_choice_df.user_id.nunique()) * 100), 1)

16.6

Вывод: пользователи хорошо понимают работу с приложением, после выбора уровня сложности к обучению обратилось 16.6% от тех кто выбрал уровень сложности